In [255]:
import numpy as np                        #importing all the required libraries
import pandas as pd 
import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/enosium-data/Track_1.csv
/kaggle/input/enosium-data/testing_data.csv


In [256]:
train = pd.read_csv('/kaggle/input/enosium-data/Track_1.csv')     #reading the dataset
test = pd.read_csv('/kaggle/input/enosium-data/testing_data.csv')

In [257]:
train = train.drop(['Unnamed: 0', '1=defaulted'], axis =1)     
test = test.drop(['Unnamed: 0'], axis =1)

In [258]:
train.isnull().sum()   #finding missing values

Number of people who will provide maintenance      0
Loan History                                       0
Purpose of taking loan                             9
loan amount taken                                  0
Guarantor or Debtor                                0
Number of years of employment                    153
Marital Status                                     0
Number of loans taken from current bank            0
Age of the applicant in Number of Years            0
amount in current account                          0
amount in savings account                          0
% of income paid as installment                    0
Other loans plans taken                            0
Working abroad or not                              0
Is there telephone number available                0
time duration for loan                             0
Owned property                                     0
Type of job performed                              0
Type of Housing                               

In [259]:
test.isnull().sum()   #finding missing values

Number of people who will provide maintenance      0
Loan History                                       0
Purpose of taking loan                             3
loan amount taken                                  0
Guarantor or Debtor                                0
Number of years of employment                    100
Marital Status                                     0
Number of loans taken from current bank            0
Age of the applicant in Number of Years            0
amount in current account                          0
amount in savings account                          0
% of income paid as installment                    0
Other loans plans taken                            0
Working abroad or not                              0
Is there telephone number available                0
time duration for loan                             0
Owned property                                     0
Type of job performed                              0
Type of Housing                               

In [260]:
print((train['Loan Defaulted or not'] == 1).sum())   #data is skewed
print((train['Loan Defaulted or not'] == 2).sum())

422
178


In [261]:
train.head()

,Number of people who will provide maintenance,Loan History,Purpose of taking loan,loan amount taken,Guarantor or Debtor,Number of years of employment,Marital Status,Number of loans taken from current bank,Age of the applicant in Number of Years,amount in current account,...,% of income paid as installment,Other loans plans taken,Working abroad or not,Is there telephone number available,time duration for loan,Owned property,Type of job performed,Type of Housing,Number of years of stay in current address,Loan Defaulted or not
0,1,critical account/other loans existing (not at ...,New Car Purchase,1028,none,between 1 and 4 years,female and divorced/seperated/married,2,36,no current account,...,4,none,Yes,No,18,Real Estate,skilled employee / official,own,3,1
1,1,existing loans paid back duly till now,Purchase of radio/television,10222,none,greater than 4 years,male and single,1,37,no current account,...,4,stores,Yes,Yes,48,car or other property,skilled employee / official,own,3,1
2,1,existing loans paid back duly till now,Purchase of radio/television,701,none,between 1 and 4 years,male and married/widowed,1,40,less than 0,...,4,none,Yes,No,12,Real Estate,unskilled - resident,own,2,1
3,1,existing loans paid back duly till now,Purchase of radio/television,1670,none,less than a year,female and divorced/seperated/married,1,22,between 0 and 200,...,4,none,Yes,Yes,9,car or other property,skilled employee / official,own,2,2
4,1,critical account/other loans existing (not at ...,New Car Purchase,1163,none,between 1 and 4 years,male and single,1,44,no current account,...,4,none,Yes,Yes,12,Real Estate,skilled employee / official,own,4,1


In [262]:
X = train.drop(["Loan Defaulted or not"], axis =1)
y = train["Loan Defaulted or not"]

In [263]:
Xt = test.drop(["Loan Defaulted or not"], axis =1)
yt = test["Loan Defaulted or not"]

In [264]:
le = LabelEncoder()                #encoding the string features
for col in  X.columns: 
        if(X[col].dtype !=X['loan amount taken'].dtype):
             X[col] = le.fit_transform(X[col])
for col in  Xt.columns:
        if(Xt[col].dtype !=Xt['loan amount taken'].dtype):
             Xt[col] = le.fit_transform(Xt[col])

In [265]:
x_train, x_val, y_train, y_val = train_test_split(X,y,test_size = 0.05, random_state =42,stratify= y)  #spliting the dataset in train and cross validation

In [266]:
from lightgbm import LGBMClassifier       #using LightBGM

lgbm = LGBMClassifier(objective='binary', random_state=42)

lgbm.fit(x_train, y_train)

y_pred_val_lightbgm = lgbm.predict(x_val)
accuracy_score(y_val, y_pred_val_lightbgm)

0.9

In [267]:
y_pred_lightbgm= lgbm.predict(Xt)

In [268]:
cat_features = list(range(0, X.shape[1]))      #using CatBoost
from catboost import CatBoostClassifier
clf = CatBoostClassifier(learning_rate=0.1,
    iterations=600,
     verbose=20,
    random_seed = 42,
)

clf.fit(
    x_train, y_train,
    cat_features=cat_features,
    eval_set=(x_val, y_val),
)

0:	learn: 0.6564711	test: 0.6555885	best: 0.6555885 (0)	total: 3.55ms	remaining: 2.13s
20:	learn: 0.4389020	test: 0.4845815	best: 0.4808805 (19)	total: 129ms	remaining: 3.56s
40:	learn: 0.3692445	test: 0.4627521	best: 0.4627521 (40)	total: 273ms	remaining: 3.71s
60:	learn: 0.3275227	test: 0.4434320	best: 0.4429929 (59)	total: 407ms	remaining: 3.59s
80:	learn: 0.2891543	test: 0.4214890	best: 0.4214890 (80)	total: 563ms	remaining: 3.61s
100:	learn: 0.2477830	test: 0.4196972	best: 0.4176829 (94)	total: 730ms	remaining: 3.61s
120:	learn: 0.2122107	test: 0.4101781	best: 0.4101781 (120)	total: 891ms	remaining: 3.52s
140:	learn: 0.1910588	test: 0.3961176	best: 0.3951510 (138)	total: 1.05s	remaining: 3.42s
160:	learn: 0.1717004	test: 0.4116620	best: 0.3951510 (138)	total: 1.19s	remaining: 3.25s
180:	learn: 0.1495682	test: 0.3967942	best: 0.3951510 (138)	total: 1.35s	remaining: 3.12s
200:	learn: 0.1340682	test: 0.3875143	best: 0.3875143 (200)	total: 1.5s	remaining: 2.98s
220:	learn: 0.1231817	t

In [269]:
y_pred_cat = clf.predict(Xt)

In [270]:
from sklearn.ensemble import AdaBoostClassifier    #using adaboost
ada = AdaBoostClassifier(n_estimators=100,learning_rate=0.1,random_state=40)
ada.fit(x_train, y_train)
y_pred_val_ada = clf.predict(x_val)
accuracy_score(y_val, y_pred_val_ada)



0.8666666666666667

In [271]:
y_pred_ada = ada.predict(Xt)

In [272]:
#ensembling 
y_ans = (y_pred_cat +y_pred_lightbgm+y_pred_ada)/3

In [273]:
for i in range(400):   #rounding off--->  1.33->1 and 1.66-> 2;
    if y_ans[i]>1.5:
        y_ans[i]=2
        i+=1
    else:
        y_ans[i] = 1
        i+=1

In [274]:
y_sub_d = pd.DataFrame(y_ans)

In [275]:
y_sub_d.index= Xt.index +600
y_sub_d.columns = ["Loan Defaulted or not"]

In [276]:
y_sub_d

,Loan Defaulted or not
600,1.0
601,1.0
602,1.0
603,1.0
604,1.0
...,...
995,1.0
996,1.0
997,1.0
998,1.0


In [278]:
y_sub_d.to_csv("ans.csv")  #converting to csv